# 05. Pytorch Going Modular

functionize the code from the previous notebook for scriptability and reusability

### 2.1 Create dataset and DataLoaders (Script Mode)

menggunakan magic function dari jupiter untuk membuat script dari cell yang diinginkan

In [ ]:
%%writefile data_setup.py
"""
Contains functionality for setting up the data for Pytorch DataLoader
"""
import os

import torchvision.transforms.v2 as trans
from torchvision import datasets
from torch.utils.data import DataLoader

num_workers = os.cpu_count()

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: trans.Compose,
    batch_size: int,
    num_workers: int=num_workers, ):
    """
    Creates training and validation DataLoader objects.
    Args:
        train_dir (str): Directory path to the training dataset.
        test_dir (str): Directory path to the validation dataset.
        transform (trans.Compose): Transformations to apply to the dataset.
        batch_size (int): Number of samples per batch.
        num_workers (int): Number of subprocesses to use for data loading.
    Returns:
        tuple: A tuple containing:
            - DataLoader: DataLoader object for the training dataset.
            - DataLoader: DataLoader object for the validation dataset.
    """
    # pake ImageFolder to create datasets
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    
    # Get class name
    class_names = train_data.classes
    
    # Turn images into DataLoader
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )
    
    return train_dataloader, test_dataloader, class_names

Overwriting data_setup.py


### 3.1 Making a model (TinyVGG) with a script

`model_builder.py`

ubah model building code into a script

In [2]:
%%writefile model_builder.py
"""
Contains functionality for building the model which is TinyVGG model from the CNN Explainer Website
"""

import torch

from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/
  
  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )
    
  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing model_builder.py


### 4.1 Turn Training function into a script

(`engine.py`)

In [ ]:
%%writefile engine.py
"""
contains functions 
"""